In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
#importing csv file while changing the client id to string
df = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv',dtype={'CLIENTNUM':str})

In [ ]:
#dropping the last two columns as per instruction on the dataset description
df = df.iloc[:,:-2]

In [ ]:
df.describe()

In [ ]:
#creating age group columns
df['age_group'] = pd.cut(df['Customer_Age'],bins=[18,26,36,46,56,66,10000000000000000000000000000],
                         labels=['A: 18-25','B: 26-35','C: 36-45','D: 46-55','E: 56-65','E: 65+'],right=False)
df['age_group'] = df['age_group'].astype(str)
df = df.drop(columns=['Customer_Age'])

In [ ]:
#changing all the columns in lower case
df.columns = df.columns.str.lower()

In [ ]:
#setting client number on index
df = df.set_index('clientnum')

In [ ]:
df['income_category'] = df['income_category'].replace({'Less than $40K':'A: Less than 40K',
                                                       '$40K - $60K':'B: 40K - 60K',
                                                       '$60K - $80K':'C: 60K - 80K',
                                                       '$80K - $120K':'D: 80K - 120K',
                                                       '$120K +':'E: 120K+'})
df.income_category.unique()

In [ ]:
df.info()

In [ ]:
print('Numeric valued columns = {}'.format(df.drop(columns=['attrition_flag']).select_dtypes(include=np.number).columns.values))

In [ ]:
print('Non-numeric valued columns = {}'.format(df.drop(columns=['attrition_flag']).select_dtypes(include='object').columns.values))      

In [ ]:
cust_count_by_attrition = pd.DataFrame(df.reset_index().groupby('attrition_flag')['clientnum'].nunique()).reset_index()
cust_count_by_attrition_fig = go.Figure([go.Pie(labels=cust_count_by_attrition['attrition_flag'], 
                                                    values=cust_count_by_attrition['clientnum'],
                                                    title="Customer Count by Attrition")])
cust_count_by_attrition_fig.update_traces(hoverinfo='label+value+percent', textinfo='percent', textfont_size=12,
                                              marker=dict(colors=['goldenrod','lightgrey'], 
                                                          line=dict(color='black', width=2)))
cust_count_by_attrition_fig.show()
cust_count_by_attrition

# **How many customers were attrited? - non-numeric valued columns only**

In [ ]:
def barplot(variable):
    att_final = pd.DataFrame(pd.crosstab(df[variable],df['attrition_flag']))
    att_final['attrition_percent'] = (att_final['Attrited Customer'] / (att_final['Attrited Customer'] + att_final['Existing Customer']) * 100).astype(int)
    
        
    color=['goldenrod','lightgrey' ]
    x=att_final.index.tolist()
    trace1 = go.Bar(x=x,
                    y=att_final['Attrited Customer'].values.tolist(),
                    name='Attrited Customer',
                    marker=dict(color='lightgrey',line=dict(color='black',width=1)))

    
    trace2 = go.Bar(x=x,
                    y=att_final['Existing Customer'].values.tolist(),
                    name='Existing Customer', 
                    marker=dict(color='goldenrod',line=dict(color='black',width=1)))
    
    trace3 =  go.Scatter(x=att_final.index,
                         y=att_final['attrition_percent'],
                         yaxis = 'y2',
                         name='Attrition %', 
                         marker=dict(color='black',line=dict(color='black',width=0.5)))

    layout = dict(title =  str(variable) + ' by Attrition Status',barmode='stack',
              xaxis=dict(title=str(variable), titlefont_size=10, tickfont_size=8,tickmode='linear'), 
              yaxis=dict(title= 'customer count', titlefont_size=10, tickfont_size=8), 
              yaxis2=dict(range= [-0, 75], overlaying= 'y', anchor= 'x', side= 'right', zeroline=False, showgrid= False, 
                          title= 'Attrition in Percent', titlefont_size=10, tickfont_size=8))

    fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
    fig.show()

In [ ]:
col_list = df.drop(columns=['attrition_flag']).select_dtypes(include='object').columns

for col in col_list:
    barplot(col)

In [ ]:
df.reset_index().pivot_table(index='card_category',columns='attrition_flag',values='clientnum', aggfunc='nunique')

# **Average (mean) figures by attrition status**

In [ ]:
np.round((pd.DataFrame(df.loc[df['attrition_flag']=='Attrited Customer'].mean()).rename(columns={0:'Attrited Customer'}).join(pd.DataFrame(df.loc[df['attrition_flag']=='Existing Customer'].mean()).rename(columns={0:'Existing Customer'}))),decimals=1)

# **Box plot distribution for numeric valued columns**

In [ ]:
col_list = df.select_dtypes(include=np.number).columns

for col in col_list:
    fig = px.box(df, y=col, color="attrition_flag",color_discrete_sequence=['goldenrod','grey']
                 ,title='Box plot distribution of ' + col)
    fig.show()

# **Histogram distribution for numeric valued columns**

In [ ]:
col_list = df.select_dtypes(include=np.number).columns


for col in col_list:
    attrited = df.loc[df['attrition_flag']=='Attrited Customer'][col].values.tolist()
    existing = df.loc[df['attrition_flag']=='Existing Customer'][col].values.tolist()

    fig = go.Figure()
    fig.add_trace(go.Histogram(x=attrited,name='Attrited Customer', marker = dict(color='grey',line=dict(color='black',width=1))))
    fig.add_trace(go.Histogram(x=existing,name='Existing Customer', marker=dict(color='goldenrod',line=dict(color='black',width=1))))

    fig.update_layout(barmode='stack', title_text='Distribution of ' + col)
    fig.show()

**Insights**

1. 93% of the customers hold blue card.
2. Revolving balance is twice as high for existing customers.
>     Recommendation: Diversification of porfolio by upgrading the cardholders.
    
3. Platinum card holders got 25% attrition rate. 
>     Recommendation: In a nutshell, VIP customers are in attrited group. Respective account managers to follow up with the cardholders, understand the customer pain points and explore the options of incentives to bring those customers back.
    
4. Existing customers did 30% more transanction than attrited customers. 
5. Attrited customers got contacted more than existing customers within past 12 months.
6. Transaction count and amount change over Q4 to Q1 doesn't tell much unless paried with more time series data.
>     Recommendation: It's important to investigate the transaction trend of attrited customers on monthly basis. Customer engagement could be essential to get the customers using the card and making transactions actively. However, the ways of communication must be re-evaluated as existing customers got contacted less (refer to point 5). Incentives such as loyalty points and voucher/coupon offers could be considered for reducing the churn rate.  
